In [ ]:
import os

import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

from plotting import set_axes_equal, set_defense_context
from utils import _pca as pca

In [ ]:
%config InlineBackend.figure_format = 'retina'

In [ ]:
def generate_random_pc(n_points=100, slope=(0.5, 0.), extent=2, noise=1.5):
    """Return a randomly distributed point cloud generated around a
    target point.
    
    Parameters
    ----------
    n_points : int, optional
        Number of points in the point cloud.
    slope : tuple, optional
        Slope of the x- and y-component of the point cloud.
    extent : number, optional
        Limits for x- and y-axis.
    noise : number, optional
        Additional noise.
    
    Returns
    -------
    tuple
        Array of points and the target point.
    """
    slope_x, slope_y = slope
    x_target = 0
    y_target = 0
    z_target = slope_x * x_target + slope_y * y_target
    target = (0, 0, z_target)
    x = np.random.normal(loc=x_target, scale=extent, size=n_points)
    y = np.random.normal(loc=y_target, scale=extent, size=n_points)
    z = slope_x * x + slope_y * y + np.random.normal(scale=noise, size=x.size)
    xyz = np.c_[x, y, z]
    return xyz, target

In [ ]:
# noisy 2-D plane in 3-D space

slope = (0.5, 0)
np.random.seed(12346789)
xyz, target = generate_random_pc(slope=slope)
target

In [ ]:
# its "clean", linearly sampled cunterpart

xs = np.linspace(xyz[:, 0].min(), xyz[:, 0].max(), 33)
ys = np.linspace(xyz[:, 1].min(), xyz[:, 1].max(), 33)
X, Y = np.meshgrid(xs, ys)
Z = slope[0] * X + slope[1] * Y

In [ ]:
# compute eigen vectors and values for clean point cloud

eigenvec, eigenval = pca(xyz)

In [ ]:
# select the unit normal as the eigen vector with the smallest eigen value

n = eigenvec[:, np.where(eigenval == eigenval.min())[0]].flatten()

In [ ]:
# extract the binormal vector and tangent vector

b = eigenvec[:, 0]
t = eigenvec[:, 1]

In [ ]:
with set_defense_context():
    fig = plt.figure(figsize=(4, 4), constrained_layout=True)
    ax = plt.axes(projection ='3d')
    
    # noisy point cloud
    ax.scatter(*xyz.T, fc='w', ec='k', label=r'$nbhd(\mathbf{x}_{i})$')
    ax.plot(*target, 'ko', mfc='r', ms=5, zorder=5, label=r'$\mathbf{x}_{i}$')
    
    # ground truth 2-D plane
    ax.plot_surface(X, Y, Z, color='k', ec='none', alpha=0.1)
    ax.text(x=xs.max(),
            y=ys.min(),
            z=slope[0] * xs.max() + slope[1] * ys.min(),
            s=r'$tp(\mathbf{x}_{i})$')
    
    # unit normal vector in target point
    ax.quiver(*target, *n, normalize=True, color='k',
              length=5, arrow_length_ratio=0.2)
    ax.text(x=n[0]-2, y=n[1], z=n[2]+3.5, s=r'$\mathbf{\hat n}$')
    
    # binormal vector
    ax.quiver(*target, *b, normalize=True, color='k',
              length=5, arrow_length_ratio=0.2)
    ax.text(x=b[0]-7.5, y=b[1]-2, z=b[2]-0.5, s=r'$\mathbf{\hat b}$')
    
    # tangent vector
    ax.quiver(*target, *t, normalize=True, color='k',
              length=5, arrow_length_ratio=0.2)
    ax.text(x=t[0]-1, y=t[1]-4, z=t[2]-0.5, s=r'$\mathbf{\hat t}$')
    ax.view_init(15, -135)
    ax.set_box_aspect([1, 1, 1])
    ax = set_axes_equal(ax)
    ax.set_axis_off()
    ax.legend(loc=1)
    plt.show()
    fname = '02-unit-normal-estimation-pca-random-pc.png'
    # fig.savefig(os.path.join('figures', fname), dpi=500, bbox_inches='tight')